# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

### Turn pdf to txt file
#### Code: 
`import glob`

`filenames = glob.glob("pdfs/*.txt")`

`contents = [open(filename, encoding="utf8").read() for filename in filenames]`

`df = pd.DataFrame({'filename': filenames, 'content': contents})`

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
# get csv from "scraped" folder
df= pd.read_csv('scraped/hm.csv')
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [3]:
# check the price colum
df['price'].head()

0      $59.99 $129
1     $6.99 $17.99
2     $7.99 $17.99
3     $6.99 $12.99
4    $17.99 $24.99
Name: price, dtype: object

In [4]:
# get dicounted price by getting all info bewtween two "$" signs
# use .astype(float) to turn str to demical
# in order to do caculations later
df['price_discounted']=df.price.str.extract(r"\$(.*)\$").astype(float)
df.head()

,name,price,url,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99


In [5]:
# get original price by getting all info after a space and the second "$"
df['price_original']=df.price.str.extract(r" \$(.*)").astype(float)
df.head()

,name,price,url,price_discounted,price_original
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99


In [6]:
df['pct_discount']=round((df['price_original'] -df['price_discounted'])/df['price_original'],2)
df.head()

,name,price,url,price_discounted,price_original,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,0.53
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,0.61
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,0.56
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,0.46
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,0.28


In [7]:
# get article id by getting info after "="
df['article_id']=df.url.str.extract(r"\=(.*)")
df.head()

,name,price,url,price_discounted,price_original,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,0.53,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,0.61,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,0.56,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,0.46,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,0.28,74242-A


In [8]:
df.to_csv("hm_cleaned.csv",index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [9]:
df1=pd.read_csv("scraped/sci-fi.csv")
df1.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [10]:
# get rid of \n in title
df1['title']=df1.title.str.strip()
df1.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness


In [11]:
# get avg_rating
df1['avg_rating']=df1['full_rating'].str.extract(r" (\d\.\d\d) a").astype(float)
df1.head()

,full_rating,full_score,rank,title,url,avg_rating
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06


In [12]:
# get rating_count
df1['rating_count']=df1['full_rating'].str.extract(r"— (.*) r")
df1.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502"
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935"
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308"
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272"
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664"


In [13]:
# get total_score
df1['total_score']=df1['full_score'].str.extract(r": (.*),")
df1.head()


,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539"
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566"
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049"
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684"
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070"


In [14]:
# get score_votes
df1['score_votes']= df1['full_score'].str.extract(r"\n(.*) p").astype(int)
df1.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165


In [15]:
# get series the book belongs to
df1['series']=df1.title.str.extract(r"[(](.*),")
df1.head(10)

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282,The Hunger Games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 people voted\n \n \n",6,"Divergent (Divergent, #1)",/book/show/13335037-divergent,4.23,"2,345,974","12,935",134,Divergent
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 people voted\n \n \n",7,"Catching Fire (The Hunger Games, #2)",/book/show/6148028-catching-fire,4.30,"2,049,239","12,261",128,The Hunger Games
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 people voted\n \n \n",8,"The Giver (The Giver, #1)",/book/show/3636.The_Giver,4.12,"1,379,452","11,238",117,The Giver
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 people voted\n \n \n",9,The Dispossessed,/book/show/13651.The_Dispossessed,4.19,"57,605","10,246",107,NaN
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people voted\n \n \n",10,Kindred,/book/show/60931.Kindred,4.20,"53,473","9,907",104,NaN


In [16]:
# get series_no
df1['series_no']=df1.title.str.extract(r", #(.*)[)]")
df1.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282,The Hunger Games,1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet,1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN,NaN


In [17]:
# save it a new csv
df1.to_csv('sci-fi_cleaned.csv',index=False)

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [18]:
# create a dataframe for all "pdf.txt" in "pdfs" folder
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
pd.set_option("display.max_colwidth", 2000)
df2 = pd.DataFrame({'filename': filenames, 'content': contents})
df2.head(3)

,filename,content
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n..."
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in..."
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\

In [19]:
# doesn't matter
# df2.content.str.strip()

Okay, now do the work and **make those new columns!**

In [20]:
# get lawyer_app, pro se means that they did it themselves

# For 1 and 11:
# FOR APPLICANT\n\nPro se\n\n

# Others have same layout:
# \n\nFor Applicant: Mark S. Zaid, Esq. \n\n


df2['lawyer_app']= df2.content.str.extract(r"for applicant:?\n?\n?(.*)\n\n",re.IGNORECASE)
df2.head()

,filename,content,lawyer_app
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...",Pro se
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",Pro se
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant f

In [21]:
# find the government's lawyer, lawyer_gov
# For 1& 11:
# n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\n
# Others looks like:
# \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\n

df2['lawyer_gov']=df2.content.str.extract(r"for government:? ?\n?\n?(.*),",re.IGNORECASE)
df2.head(3)

,filename,content,lawyer_app,lawyer_gov
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...",Pro se,"David F. Hayes, Esq."
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",Pro se,"James B. Norman, Esq."
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND 

In [22]:
# get names of the judge
# 1 & 11:
# \nJames E. Moody\nAdministrative Judge\nMember, Appeal Board\n\n
# Others:
# \n\nMENDEZ, Francisco, Administrative Judge: \n\n

df2['judge']= df2.content.str.extract(r"\n(.*),?\n?Administrative Judge:? ?\n",re.IGNORECASE)
df2.head(3)

,filename,content,lawyer_app,lawyer_gov,judge
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...",Pro se,"David F. Hayes, Esq.","MENDEZ, Francisco,"
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",Pro se,"James B. Norman, Esq.",Michael Ra’anan
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF D

In [23]:
# access
# For 1 & 11:
# \n\nThe Decision is AFFIRMED.  \n\n
# Others:
#\n\naccess to classified information is granted. \n \n

### 2 NaN:
# for 0 
# Clearance is denied
# for 3
# Clearance is granted


df2['acess']= df2.content.str.extract(r"ion is (.*). \n",re.IGNORECASE)
df2.head()

,filename,content,lawyer_app,lawyer_gov,judge,acess
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...",Pro se,"David F. Hayes, Esq.","MENDEZ, Francisco,",NaN
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",Pro se,"James B. Norman, Esq.",Michael Ra’anan,AFFIRMED.
2,pdfs/11-03073.h1.pdf.txt,"

In [24]:
# BONUS: try debt
df2['has_debt']=df2.content.str.contains(r"debt")
df2.head(3)

,filename,content,lawyer_app,lawyer_gov,judge,acess,has_debt
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...",Pro se,"David F. Hayes, Esq.","MENDEZ, Francisco,",NaN,False
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",Pro se,"James B. Norman, Esq.",Michael Ra’anan,AFFIRMED.,True
2,pdfs/1

In [25]:
# has_bankruptcy
df2['has_bankruptcy']=df2.content.str.contains(r"bankruptcy")
df2.head(3)

,filename,content,lawyer_app,lawyer_gov,judge,acess,has_debt,has_bankruptcy
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...",Pro se,"David F. Hayes, Esq.","MENDEZ, Francisco,",NaN,False,False
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",Pro se,"James B. Norman, Esq.",Michael Ra’anan,AF

In [26]:
# save it a new csv
df2.to_csv('court_cleaned.csv',index=False)

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [27]:
# encoding="utf8"
contents = open("data/Austen_Pride.txt",encoding="utf8").read()
#contents

## Look at the first 500 or so characters of it 

In [28]:
contents[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [29]:
# find all she and he
he_she=re.findall(r"\bs?he\b",contents,re.IGNORECASE)
he_she
#len(he_she)

['he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'She',
 'she',
 'she',
 'He',
 'he',
 'she',
 'he',
 'She',
 'She',
 'she',
 'she',
 'she',
 'he',
 'he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'She',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'She',
 'he',
 'he',
 'She',
 'She',
 'she',
 'she',
 'he',
 'he',
 'He',
 'he',
 'he',
 'she',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'she',
 'She',
 'she',
 'he',
 'He',
 'he',
 'she',
 'He',
 'she',
 'He',
 'he',
 'He',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [30]:
# find she.he + a word
he_she_word=re.findall(r"\bs?he\b \w*",contents,re.IGNORECASE)
he_she_word[0:5]
#len(he_she_word)

['he is', 'he had', 'she told', 'he came', 'he agreed']

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [31]:
groups=re.findall(r"(\bs?he\b) (\w*)",contents,re.IGNORECASE)
groups[0:5]

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [32]:
columns=[]
for word in groups:
    my_dic={}
    my_dic['pronoun']=word[0]
    my_dic['verb']= word[1]
    columns.append(my_dic)
columns[0:5]

[{'pronoun': 'he', 'verb': 'is'},
 {'pronoun': 'he', 'verb': 'had'},
 {'pronoun': 'she', 'verb': 'told'},
 {'pronoun': 'he', 'verb': 'came'},
 {'pronoun': 'he', 'verb': 'agreed'}]

In [33]:
df3 = pd.DataFrame(columns)
df3.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [34]:
df3.pronoun.value_counts()

she    1324
he     1056
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [35]:
df3['pronoun']=df3['pronoun'].str.lower()
df3.pronoun.value_counts()

she    1649
he     1290
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [36]:
df3['verb'].value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
thought       31
added         31
then          26
replied       22
continued     21
looked        21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [37]:
df3[df3['pronoun']=='he']['verb'].value_counts().head(20)

had        166
was        160
is          74
has         49
could       40
would       35
did         29
should      26
will        24
must        24
might       21
replied     14
thought     13
may         13
came        13
said        13
never       12
does        12
meant       11
then        11
Name: verb, dtype: int64

In [38]:
df3[df3['pronoun']=='she']['verb'].value_counts().head(20)

was        212
had        205
could      132
is          65
would       59
did         38
felt        33
saw         29
will        26
might       25
added       23
has         21
said        20
thought     18
then        15
should      15
must        13
found       13
soon        12
spoke       11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [39]:
df3[df3['verb']=='cried']['pronoun'].value_counts(normalize=True)

she    0.916667
he     0.083333
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [40]:
contents2 = open("data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt",encoding="utf8").read()
contents2[:500]

'THE FELLOWSHIP OF THE RING\n\n\n\n\nBEING THE FIRST PART OF\n\nTHE LORD OF THE RINGS\n\nBY\n\nJ.R.R. TOLKIEN\n\n\n\n\n\nThree Rings for the Elven-kings under the sky,\n\nSeven for the Dwarf-lords in their halls of stone,\n\nNine for Mortal Men doomed to die,\n\nOne for the Dark Lord on his dark throne\n\nIn the Land of Mordor where the Shadows lie.\n\nOne Ring to rule them all, One Ring to find them,\n\nOne Ring to bring them all and in the darkness bind them\n\nIn the Land of Mordor where the Shadows lie.\n\n\n\n\n\nCONTENTS\n\n\nCOV'

In [41]:
# find out he/she and a word

groups2=re.findall(r"(\bs?he\b) (\w*)",contents2,re.IGNORECASE)
groups2[0:5]

[('he', 'had'),
 ('He', 'was'),
 ('he', 'wrote'),
 ('he', 'recorded'),
 ('he', 'had')]

In [42]:
columns2=[]
for word in groups2:
    my_dic={}
    my_dic['pronoun']=word[0]
    my_dic['verb']= word[1]
    columns2.append(my_dic)
# columns2[0:5]
df4 = pd.DataFrame(columns2)
df4.head()

,pronoun,verb
0,he,had
1,He,was
2,he,wrote
3,he,recorded
4,he,had


In [43]:
df4['pronoun']=df4['pronoun'].str.lower()
df4.pronoun.value_counts()

he     2986
she     156
Name: pronoun, dtype: int64

In [44]:
# The percentage of "he" in L.R.R is huge!
df4.pronoun.value_counts(normalize=True)

he     0.95035
she    0.04965
Name: pronoun, dtype: float64

In [45]:
# compare to Pride and Prejudice txt
df3.pronoun.value_counts(normalize=True)

she    0.561075
he     0.438925
Name: pronoun, dtype: float64